Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>=DESIRED_ACCURACY):
              print("\nReached 99.9% accuracy so cancelling training!")
              self.model.stop_training = True

callbacks = myCallback()

--2019-12-18 16:55:18--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2019-12-18 16:55:19 (202 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

    # Please use a target_size of 150 X 150.
train_generator = train_datagen.flow_from_directory( '/tmp/h-or-s',  # This is the source directory for training images
target_size=(150, 150),  # All images will be resized to 150x150
batch_size=80,
        # Since we use binary_crossentropy loss, we need binary labels
class_mode='binary')
    # Expected output: 'Found 80 images belonging to 2 classes'


Found 80 images belonging to 2 classes.


In [6]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(train_generator,
                                  steps_per_epoch=8,
                                  epochs=15,
                                  verbose=1,
                                  callbacks=[callbacks])
    # model fitting
return history.history['acc'][-1]    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 5s 679ms/step - loss: 1.0974 - acc: 0.6031
Epoch 2/15
8/8 [==============================] - 2s 202ms/step - loss: 0.3359 - acc: 0.8672
Epoch 3/15
8/8 [==============================] - 2s 200ms/step - loss: 0.1463 - acc: 0.9625
Epoch 4/15
8/8 [==============================] - 2s 191ms/step - loss: 0.0983 - acc: 0.9750
Epoch 5/15
8/8 [==============================] - 2s 198ms/step - loss: 0.0741 - acc: 0.9797
Epoch 6/15
8/8 [==============================] - 2s 188ms/step - loss: 0.0287 - acc: 0.9937
Epoch 7/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0095 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 2s 200ms/step - loss: 0.0091 - acc: 1.0000


SyntaxError: ignored